In [1]:
import re
import os
import json
import gzip
import tarfile
import itertools as itt

from xml.etree import ElementTree
from tqdm import tqdm_notebook as tqdm

In [13]:
ARTICLES = ["./round2/biorxiv_medrxiv.tar.gz.tar.gz", "./round2/noncomm_use_subset.tar.gz"]
METADATA = "./round2/metadata.csv"
TOPICS = "./round2/topics-rnd2.xml"
VALID_DOCS = "./round2/docids-rnd2.txt"
QRELS = "./round2/qrels-rnd2.txt"

In [65]:
class Topic():
    def __init__(self, index, query, question, narrative):
        self.index = index
        self.query = query
        self.question = question
        self.narrative = narrative
    
    def __str__(self):
        return self.index + " " + self.query

In [66]:
class TopicCollection():    
    def __init__(self, path):
        self.topics = {}
        tree = ElementTree.parse(path)        
        for child in tree.getroot():
            n = child.attrib['number']
            query = child.find('query').text
            question = child.find('question').text
            narrative = child.find('narrative').text
            self.topics[int(n)] = Topic(n, query, question, narrative)
     
    def __length__(self):
        return len(self.topics)
    
    def get_topic(self, idx):
        return self.topics[idx]

In [67]:
t = TopicCollection(TOPICS)

In [68]:
t.__length__()

35

In [69]:
i = t.get_topic(1)
print(i)

1 coronavirus origin
